In [4]:
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML

test ="""
        #Game No : 105553145604320
        ***** Hand History for Game 23093091932 *****
        €2 EUR PL Omaha - Monday,October 17, 21:26:03 CEST 2022
        Table Las Palmas de G. C. ( Real Money )
        Seat 4 is the button
        Total number of players : 6/6
        seat 1: jejesat ( €2 EUR )
        seat 2: Player2 ( €3.81 EUR )
        seat 3: Player3 ( €0.79 EUR )
        seat 4: Player4 ( €7.92 EUR )
        seat 5: Player5 ( €2 EUR )
        seat 6: Player6 ( €1.85 EUR )
        Player6 posts small blind [€0.01 EUR].
        jejesat posts big blind [€0.02 EUR].
        ** Dealing down cards **
        Dealt to jejesat [ Ah, 5h, 7d, Jd ]
        Caipirinha00 folds
        Kcila folds
        sankukai99 folds
        pomlau will be using their time bank for this hand.
        pomlau calls [ €0.01 EUR ]
        jejesat raises [ €0.04 EUR ]
        pomlau calls [ €0.04 EUR ]
        ** Dealing Flop ** [ Td, Ts, 2c ]
        pomlau checks
        jejesat bets [ €0.06 EUR ]
        pomlau folds
        jejesat does not show cards.
        jejesat wins €0.18 EUR
        """


substitutions = {
                     'LEGAL_ISO' : "USD|EUR",            # legal ISO currency codes
                            'LS' : u"\$|\u20ac|\xe2\x82\xac|",    # Currency symbols - Euro(cp1252, utf-8)
                           'NUM' : u".,'\dKMB",
                    }

re_GameInfo = re.compile(u"""
            \*{5}\sHand\sHistory\s(F|f)or\sGame\s(?P<HID>\w+)\s\*{5}(\s\((?P<SITE>Poker\sStars|PokerMaster|Party|PartyPoker|IPoker|Pacific|WPN|PokerBros)\))?\s+
            (.+?\shas\sleft\sthe\stable\.\s+)*
            (.+?\sfinished\sin\s\d+\splace\.\s+)*
            ((?P<CURRENCY>[%(LS)s]))?\s*
            (
             ([%(LS)s]?(?P<SB>[%(NUM)s]+)/[%(LS)s]?(?P<BB>[%(NUM)s]+)\s*(?:%(LEGAL_ISO)s)?\s+(?P<FAST3>fastforward\s)?((?P<LIMIT3>NL|PL|FL|)\s+)?)|
             ((?P<CASHBI>[%(NUM)s]+)\s*(?:%(LEGAL_ISO)s)?\s*)(?P<FAST2>fastforward\s)?(?P<LIMIT2>(NL|PL|FL|))?\s*
            )
            (Tourney\s*)?
            (?P<GAME>(Texas\sHold\'?em|Hold\'?em|Omaha\sHi-Lo|Omaha(\sHi)?|7\sCard\sStud\sHi-Lo|7\sCard\sStud|Double\sHold\'?em|Short\sDeck))\s*
            (Game\sTable\s*)?
            (
             (\((?P<LIMIT>(NL|PL|FL|Limit|))\)\s*)?
             (\((?P<SNG>SNG|STT|MTT)(\sJackPot)?\sTournament\s\#(?P<TOURNO>\d+)\)\s*)?
            )?
            (?:\s\(Buyin\s(?P<BUYIN>[%(LS)s][%(NUM)s]+)\s\+\s(?P<FEE>[%(LS)s][%(NUM)s]+)\))?
            \s*-\s*
            (?P<DATETIME>.+)
            """ % substitutions, re.VERBOSE | re.UNICODE)


In [5]:
m = re_GameInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,HID,SITE,CURRENCY,SB,BB,FAST3,LIMIT3,CASHBI,FAST2,LIMIT2,GAME,LIMIT,SNG,TOURNO,BUYIN,FEE,DATETIME
0,23093091932,None,€,None,None,None,None,2,None,PL,Omaha,None,None,None,None,None,"Monday,October 17, 21:26:03 CEST 2022"


In [6]:
re_HandInfo = re.compile(u"""
            Table\s(?P<TABLE>.+?)?\s+
            ((?: \#|\(|)(?P<TABLENO>\d+)\)?\s+)?
            (\(No\sDP\)\s)?
            \(\s?(?P<PLAY>Real|Play)\s+Money\s?\)\s+(--\s*)? # FIXME: check if play money is correct
            Seat\s+(?P<BUTTON>\d+)\sis\sthe\sbutton
            (\s+Total\s+number\s+of\s+players\s+\:\s+(?P<PLYRS>\d+)/?(?P<MAX>\d+)?)?
            """, re.VERBOSE|re.MULTILINE|re.DOTALL)

In [7]:
m = re_HandInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,TABLE,TABLENO,PLAY,BUTTON,PLYRS,MAX
0,Las Palmas de G. C.,None,Real,4,6,6


In [8]:
re_GameInfoTrny1     = re.compile(u"""
            \*{5}\sHand\sHistory\s(F|f)or\sGame\s(?P<HID>\w+)\s\*{5}\s+
            (?P<LIMIT>(NL|PL|FL|))\s*
            (?P<GAME>(Texas\sHold\'em|Hold\'?em|Omaha\sHi-Lo|Omaha(\sHi)?|7\sCard\sStud\sHi-Lo|7\sCard\sStud|Double\sHold\'em|Short\sDeck))\s+
            (?:(?P<BUYIN>[%(LS)s]?\s?[%(NUM)s]+)\s*(?P<BUYIN_CURRENCY>%(LEGAL_ISO)s)?\s*Buy-in\s+)?
            (\+\s(?P<FEE>[%(LS)s]?\s?[%(NUM)s]+)\sEntry\sFee\s+)?
            Trny:\s?(?P<TOURNO>\d+)\s+
            Level:\s*(?P<LEVEL>\d+)\s+
            ((Blinds|Stakes)(?:-Antes)?)\(
                (?P<SB>[%(NUM)s ]+)\s*
                /(?P<BB>[%(NUM)s ]+)
                (?:\s*-\s*(?P<ANTE>[%(NUM)s ]+)\$?)?
            \)
            \s*\-\s*
            (?P<DATETIME>.+)
            """ % substitutions, re.VERBOSE | re.UNICODE)

In [9]:
m = re_GameInfoTrny1.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [10]:
re_GameInfoTrny2     = re.compile(u"""
            \*{5}\sHand\sHistory\s(F|f)or\sGame\s(?P<HID>\w+)\s\*{5}\s+
            (?P<LIMIT>(NL|PL|FL|))\s*
            (?P<GAME>(Texas\sHold\'em|Hold\'?em|Omaha\sHi-Lo|Omaha(\sHi)?|7\sCard\sStud\sHi-Lo|7\sCard\sStud|Double\sHold\'em|Short\sDeck))\s+
            (?:(?P<BUYIN>[%(LS)s]?\s?[%(NUM)s]+)\s*(?P<BUYIN_CURRENCY>%(LEGAL_ISO)s)?\s*Buy-in\s+)?
            (\+\s(?P<FEE>[%(LS)s]?\s?[%(NUM)s]+)\sEntry\sFee\s+)?
            \s*\-\s*
            (?P<DATETIME>.+)
            """ % substitutions, re.VERBOSE | re.UNICODE)

In [11]:
m = re_GameInfoTrny2.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [12]:
re_GameInfoTrny3     = re.compile(u"""
            \*{5}\sHand\sHistory\s(F|f)or\sGame\s(?P<HID>\w+)\s\*{5}\s\((?P<SITE>Poker\sStars|PokerMaster|Party|IPoker|Pacific|WPN|PokerBros)\)\s+
            Tourney\sHand\s
            (?P<LIMIT>(NL|PL|FL|))\s*
            (?P<GAME>(Texas\sHold\'em|Hold\'?em|Omaha\sHi-Lo|Omaha(\sHi)?|7\sCard\sStud\sHi-Lo|7\sCard\sStud|Double\sHold\'em|Short\sDeck))\s+
            \s*\-\s*
            (?P<DATETIME>.+)
            """ % substitutions, re.VERBOSE | re.UNICODE)

In [13]:
m = re_GameInfoTrny3.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [14]:
re_Blinds = re.compile("""
            ((Blinds|Stakes)(?:-Antes)?)\(
                (?P<SB>[%(NUM)s ]+)\s*
                /(?P<BB>[%(NUM)s ]+)
                (?:\s*-\s*(?P<ANTE>[%(NUM)s ]+)\$?)?
            \)$""" % substitutions, re.VERBOSE | re.MULTILINE)

In [15]:
m = re_Blinds.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [16]:
re_TourNoLevel = re.compile("""
            Trny:\s?(?P<TOURNO>\d+)\s+
            Level:\s*(?P<LEVEL>\d+)
        """, re.VERBOSE)

In [17]:
m = re_TourNoLevel.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [18]:
re_PlayerInfo   = re.compile(u"""
          (S|s)eat\s?(?P<SEAT>\d+):\s
          (?P<PNAME>.*)\s
          \(\s*[%(LS)s]?(?P<CASH>[%(NUM)s]+)\s*(?:%(LEGAL_ISO)s|)\s*\)
          """ % substitutions, re.VERBOSE| re.UNICODE)

In [19]:
m = re_PlayerInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,SEAT,PNAME,CASH
0,1,jejesat,2


In [20]:
re_NewLevel = re.compile(u"Blinds(-Antes)?\((?P<SB>[%(NUM)s ]+)/(?P<BB>[%(NUM)s ]+)(?:\s*-\s*(?P<ANTE>[%(NUM)s ]+))?\)" % substitutions, re.VERBOSE|re.MULTILINE|re.DOTALL)

In [21]:
m = re_NewLevel.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [22]:
re_CountedSeats = re.compile("Total\s+number\s+of\s+players\s*:\s*(?P<COUNTED_SEATS>\d+)", re.MULTILINE)

In [23]:
m = re_CountedSeats.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,COUNTED_SEATS
0,6


In [24]:
re_Identify     = re.compile(u'\*{5}\sHand\sHistory\s[fF]or\sGame\s\d+\w+?\s')

In [25]:
m = re_Identify.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

""


In [26]:
re_Button       = re.compile('Seat (?P<BUTTON>\d+) is the button', re.MULTILINE)

In [28]:
m = re_Button.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,BUTTON
0,4


In [29]:
re_Board        = re.compile(r"\[(?P<CARDS>.+)\]")

In [30]:
m = re_Board.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,CARDS
0,€0.01 EUR


In [33]:
 re_NoSmallBlind = re.compile(
                    '^There is no Small Blind in this hand as the Big Blind '
                    'of the previous hand left the table', re.MULTILINE)
m = re_NoSmallBlind.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))